
# Machine Learning Based Recommendation Systems
##  Model-based Collaborative Filtering Systems
## SVD Matrix Factorization

In [23]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import TruncatedSVD

The MovieLens dataset was collected by the GroupLens Research Project at the University of Minnesota. You can download the dataset for this demostration at the following URL: https://grouplens.org/datasets/movielens/100k/

### Preparing the data

In [24]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
frame = pd.read_csv('ml-100k/u.data', sep='\t', names=columns)
frame.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [67]:
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('ml-100k/u.item', sep='|', names=columns, encoding='latin-1')
movie_names = movies[['item_id', 'movie title']]
movie_names.head()

,item_id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [26]:
combined_movies_data = pd.merge(frame, movie_names, on='item_id')
combined_movies_data.head()

,user_id,item_id,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [27]:
combined_movies_data.groupby('item_id')['rating'].count().sort_values(ascending=False).head()

item_id
50     583
258    509
100    508
181    507
294    485
Name: rating, dtype: int64

In [68]:
r = pd.DataFrame(combined_movies_data.groupby('item_id')['rating'].sum())
movie_names['rating'] = r['rating']

/Users/nrd1012/.pyenv/versions/3.4.3/envs/nrbase/lib/python3.4/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [69]:
movie_names

,item_id,movie title,rating
0,1,Toy Story (1995),NaN
1,2,GoldenEye (1995),1753.0
2,3,Four Rooms (1995),420.0
3,4,Get Shorty (1995),273.0
4,5,Copycat (1995),742.0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,284.0
6,7,Twelve Monkeys (1995),93.0
7,8,Babe (1995),1489.0
8,9,Dead Man Walking (1995),875.0
9,10,Richard III (1995),1165.0


In [28]:
filter = combined_movies_data['item_id']==50
combined_movies_data[filter]['movie title'].unique()

array(['Star Wars (1977)'], dtype=object)

### Building a Utility Matrix

In [29]:
# combined_movies_data['rating'].fillna(value=0, inplace=True)

# um = pd.pivot_table(data=combined_movies_data, values='rating', index='user_id', columns='movie title')
# um.fillna(value=0, inplace=True)

um = pd.pivot_table(data=combined_movies_data, values='rating', index='user_id', columns='movie title', fill_value=0)
um.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,0,0,0,5,3,0,0,0,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,4,0,0,0,0,4,0


### Transposing the Matrix

In [30]:
um.shape

(943, 1664)

In [31]:
X = um.T
X.shape

(1664, 943)

### Decomposing the Matrix

In [32]:
SVD = TruncatedSVD(n_components=12, random_state=17)
resultant_matrix = SVD.fit_transform(X)
resultant_matrix.shape

(1664, 12)

### Generating a Correlation Matrix

In [33]:
resultant_matrix[0]

array([ 1.03999361,  0.6598845 ,  0.04568944,  0.81444724,  0.16747427,
       -0.98316366, -0.3198289 ,  0.3067836 , -0.03084219, -0.75356126,
       -0.30152968, -0.53651145])

In [39]:
corr_mat = pd.DataFrame(resultant_matrix.T).corr()
corr_mat.shape
corr_mat.values

array([[ 1.        , -0.10298113,  0.52210159, ...,  0.39854553,
         0.22143017,  0.5039286 ],
       [-0.10298113,  1.        ,  0.06549218, ...,  0.16134137,
         0.5091753 ,  0.23355053],
       [ 0.52210159,  0.06549218,  1.        , ...,  0.7658073 ,
         0.44348034,  0.19721751],
       ..., 
       [ 0.39854553,  0.16134137,  0.7658073 , ...,  1.        ,
         0.18088492,  0.10342131],
       [ 0.22143017,  0.5091753 ,  0.44348034, ...,  0.18088492,
         1.        ,  0.18524109],
       [ 0.5039286 ,  0.23355053,  0.19721751, ...,  0.10342131,
         0.18524109,  1.        ]])

In [42]:
corr_mat = np.corrcoef(resultant_matrix)

### Isolating Star Wars From the Correlation Matrix

In [52]:
movie_names = um.columns
movies_list = list(movie_names)
star_wars = movies_list.index('Star Wars (1977)')
star_wars

1398

In [50]:
from sklearn.externals import joblib
joblib.dump(corr_mat, '/Users/nrd1012/Projects/sid/winerec_tutorial/winerama/tutorial/rec_model.pkl') 
joblib.dump(movie_names, '/Users/nrd1012/Projects/sid/winerec_tutorial/winerama/tutorial/movie_names.pkl')

['/Users/nrd1012/Projects/sid/winerec_tutorial/winerama/tutorial/movie_names.pkl']

In [45]:
corr_star_wars = corr_mat[1398]
corr_star_wars.shape

(1664,)

### Recommending a Highly Correlated Movie

['Die Hard (1988)',
 'Empire Strikes Back, The (1980)',
 'Fugitive, The (1993)',
 'Raiders of the Lost Ark (1981)',
 'Return of the Jedi (1983)',
 'Terminator 2: Judgment Day (1991)',
 'Terminator, The (1984)',
 'Toy Story (1995)']

In [49]:
type(movie_names)

pandas.core.indexes.base.Index

In [47]:
list(movie_names[(corr_star_wars<1.0) & (corr_star_wars > 0.9)])

['Die Hard (1988)',
 'Empire Strikes Back, The (1980)',
 'Fugitive, The (1993)',
 'Raiders of the Lost Ark (1981)',
 'Return of the Jedi (1983)',
 'Terminator 2: Judgment Day (1991)',
 'Terminator, The (1984)',
 'Toy Story (1995)']

In [48]:
list(movie_names[(corr_star_wars<1.0) & (corr_star_wars > 0.95)])

['Return of the Jedi (1983)']